In [ ]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

elastic_rods.set_max_num_tbb_threads(4)
l = elastic_rods.RodLinkage('../../../examples/florin/test11.obj', 5)
driver=260
l.setMaterial(elastic_rods.RodMaterial('+', 2000, 0.3, [2, 2, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1))

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1800, height=1024)
view.setCameraParams(((1.4948114201545684, -1.4790700211130228, -0.46246701187663153),
 (0.1395482312739102, -0.22435052911050535, -0.9644652047825047),
 (0.04574869063383073, -0.017577456701375806, -0.17030459030516723)))
view.show()

In [ ]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_test11.txt', zeroBasedIndexing=True)
np.savetxt('restlen_test11.txt',l.getPerSegmentRestLength())

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi / 3, 10, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(93) - l.averageJointAngle, 10, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=10, useTargetAngleConstraint=True);